In [1]:
import xarray 
import os
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import imdlib as imd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
import matplotlib.image as mpimg
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import random


In [3]:
x=pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 1901-2022.csv",index_col=0)
y=pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\Rainfall Clusters 1901-2022.csv",index_col=0)

In [4]:
value_counts=y.value_counts().sort_index()
value_counts

Cluster Number
0                 12
1                  9
2                 11
3                 39
4                 14
5                  5
6                 12
7                  6
8                 14
dtype: int64

In [5]:
# Find the cluster number with the most points
most_common_cluster = value_counts.idxmax()
most_common_cluster

(3,)

In [6]:
# Filter the DataFrame to select only the rows with the most common cluster
most_common_cluster_rows = y[y['Cluster Number'] == most_common_cluster]

cluster_3_indexes=list(most_common_cluster_rows.index)
print("Cluster 3 total years",len(cluster_3_indexes))



cluster_3_dropping_indexes = random.sample(cluster_3_indexes, (2*len(cluster_3_indexes))//3)

print("\nUsing years ",cluster_3_dropping_indexes)
print("\nLenght of dropping years",len(cluster_3_dropping_indexes))

Cluster 3 total years 39

Using years  [29, 91, 45, 104, 92, 88, 53, 12, 22, 30, 28, 54, 96, 13, 56, 120, 36, 42, 59, 11, 27, 48, 31, 49, 89, 39]

Lenght of dropping years 26


In [7]:
x.drop(cluster_3_dropping_indexes,inplace=True)
y.drop(cluster_3_dropping_indexes,inplace=True)

In [9]:
value_counts=y.value_counts().sort_index()
value_counts

Cluster Number
0                 12
1                  9
2                 11
3                 13
4                 14
5                  5
6                 12
7                  6
8                 14
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

In [13]:
from sklearn import tree


model=tree.DecisionTreeRegressor()
model.fit(x_train,y_train)

DecisionTreeRegressor()

In [14]:
y_pred = model.predict(x_test)
y_pred

array([6., 4., 4., 0., 1., 6., 8., 4., 1., 8.])

In [15]:
y_array=y_test.values.ravel()
y_array

array([8, 5, 4, 4, 4, 6, 4, 1, 2, 2], dtype=int64)

In [16]:
accuracy_score(y_pred,y_array)

0.2

In [17]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [18]:
model_params = {
    'svm': {
        'model': svm.SVC(random_state=42),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear'],
            'degree': [2, 3, 4],
            'coef0': [0.0, 0.1, 0.5],
            'probability': [True, False],
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [1, 5, 10, 20, 50],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear',random_state=42,max_iter=2000),
        'params': {
            'C': [1, 5, 10],
            'penalty': ['l1', 'l2'],
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.1, 0.5, 1.0, 2.0],
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }
    },
    'linear_discriminant_analysis': {
        'model': LinearDiscriminantAnalysis(),
        'params' : {

        }
    },
}



In [20]:
y_train_array=y_train.values.ravel()
len(y_train_array)

86

In [21]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train_array)
    y_pred = clf.predict(x_test)
    y_pred = np.ceil(y_pred)
    y_test_array= y_test.values.ravel()
    y_test_array
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'accuracy_score': accuracy_score(y_pred,y_test_array),
        'best_params': clf.best_params_,
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','accuracy_score','best_params'])
df

C:\Users\ratho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\ratho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\ratho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\ratho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\ratho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\ratho\anaconda3\lib\s

,model,best_score,accuracy_score,best_params
0,svm,0.103922,0.0,"{'C': 10, 'coef0': 0.0, 'degree': 2, 'kernel':..."
1,random_forest,0.173856,0.0,"{'criterion': 'gini', 'max_depth': None, 'min_..."
2,logistic_regression,0.104575,0.0,"{'C': 1, 'penalty': 'l1'}"
3,naive_bayes_gaussian,0.035294,0.1,{}
4,naive_bayes_multinomial,0.069935,0.0,{'alpha': 0.1}
5,decision_tree,0.115686,0.1,"{'criterion': 'entropy', 'max_depth': None, 'm..."
6,linear_discriminant_analysis,0.081046,0.0,{}
